# MNIST (필기체 숫자) 인식

In [1]:
import numpy as np

In [2]:
# 시그모이드 함수
def sigmoid(x):
    return 1/(1+np.exp(-x))



# 수치미분 함수
def numerical_derivative(f,x): # x에 변수가 여러개
    delta_x = 1e-4
    grad = np.zeros_like(x)
    # 주어진 어레이와 같은 형태와 타입을 갖는 0으로 채워진 어레이를 반환
    # grad는 계산된 수치미분 값을 저장할 변수
    it = np.nditer(x,flags=['multi_index'],op_flags=['readwrite'])
    #  op_flags를 이용하면 기본값은 읽기 전용이지만 읽기/쓰기 또는 
    # 쓰기 전용 모드로 설정할 수 있다
    # multi_index는 뭐지..?
    while not it.finished:
        idx = it.multi_index
        
        tmp_val = x[idx]
        # x의 원본값이 변화되는 것을 막고자 임시변수 tmp_val 이라는 곳에
        # 원본값을 저장해두고 있다
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_X)
        
        x[idx]=tmp_val - delta_x
        fx2 = f(x) # f(x-delta_X)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val
        it.iternext()
    
    return grad

In [12]:
class NeuralNetwork:
    #생성자 
    def __init__(self,input_nodes,hidden_nodes,output_nodes):
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes
        
        # 2층 hidden layer unit
        # 가중치 W, 바이어스 b 초기화
        self.W2 = np.random.rand(self.input_nodes,self.hidden_nodes) # W2 = (784X100)
        self.b2 = np.random.rand(self.hidden_nodes)                  # b2 = (100,)
        
        # 3층 output layer unit
        self.W3 = np.random.rand(self.hidden_nodes,self.output_nodes) # W3 = (100X10)
        self.b3 = np.random.rand(self.output_nodes)                   # b3 = (10,)
        
        self.learning_rate = 1e-4
        
    def feed_forward(self):
        delta = 1e-7
        
        z1 = np.dot(self.input_data, self.W2) + self.b2
        y1 = sigmoid(z1)
        
        z2 = np.dot(y1,self.W3) + self.b3
        y = sigmoid(z2)
        
        #cross-entropy
        return -np.sum(self.target_data*np.log(y+delta)+(1-self.target_data)*np.log((1-y)+delta))
    
    def loss_val(self):
        
        delta = 1e-7
        
        z1 = np.dot(self.input_data, self.W2) + self.b2
        y1 = sigmoid(z1)
        
        z2 = np.dot(y1,self.W3) + self.b3
        y = sigmoid(z2)
        
        #cross-entropy
        return -np.sum(self.target_data*np.log(y+delta)+(1-self.target_data)*np.log((1-y)+delta))
    
    
    def train(self, training_data):
            
            #one-hot encoding을 위해 10개의 노드 0.01로 초기화 및 정답 데이터는 0.99로 초기화
            self.target_data = np.zeros(output_nodes)+0.01
            self.target_data[int(training_data[0])] = 0.99
            
            #입력데이터가 0~255 사이이기 때문에, 가끔 overflow 발생하므로 모드 입력값을 
            #0~1 사이의 값으로 normalize 함
            #데이터의 최댓값을 나눠줌
            self.input_data=(training_data[1:]/255.0*0.99)+ 0.01
            
            f = lambda x : self.feed_forward()
                
            self.W2 -= self.learning_rate * numerical_derivative(f,self.W2)
            self.b2 -= self.learning_rate * numerical_derivative(f,self.b2)
            self.W3 -= self.learning_rate * numerical_derivative(f,self.W3)
            self.b3 -= self.learning_rate * numerical_derivative(f,self.b3)
    
    def predict(self,input_data):
        
        z1 = np.dot(input_data,self.W2)+self.b2
        y1 = sigmoid(z1)
        
        z2 = np.dot(y1,self.W3) + self.b3
        y = sigmoid(z2)
        
        # 가장 큰 값을 가지는 인덱스를 정답으로 인식함
        # one-hot endcoding
        predicted_num = np.argmax(y)
        
        return predicted_num
    
    #정확도 측정함수
    def accuracy(self, test_data):
        matched_list =[]
        not_matched_list = []
        
        for index in range(leng(test_data)):
            label = int(test_data[index,0])
            
            #normalize
            data = (test_data[index,1:]/255.0*0.99)+0.01
            
            predicted_num = self.predict(data)
            
            if label == predicted_num:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
        print("Current Accuracy = ", 100*(len(matched_list)/len(test_data))," %")
        
        return matched_list, not_matched_list

### MNIST (필기체 숫자 인식) 검증

In [13]:
training_data = np.loadtxt('./mnist_train.csv',delimiter=',',dtype=np.float32)
test_data = np.loadtxt('./mnist_test.csv',delimiter=',',dtype=np.float32)

input_nodes = 784
hidden_nodes = 100
output_nodes = 10

mn = NeuralNetwork(input_nodes,hidden_nodes, output_nodes)

for step in range(30001): #전체 training data의 50%
    index = np.random.randint(0,len(training_data)-1)
    mn.train(training_data[index])
    
    if step%400 == 0:
        print("step = ",step,", loss_val =",mn.loss_val())

step =  0 , loss_val = 143.77341309854825
step =  400 , loss_val = 143.77341309854825


KeyboardInterrupt: 

In [ ]:
# accuracy 계산
mn.accuracy(test_data)